In [5]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import count,col, when,countDistinct,mean

spark = SparkSession.builder.getOrCreate()

In [8]:
df_dpe = spark.read.option("header",True).csv("./work/data/dpe-v2-logements-existants.csv")
df_dpe.show(5) 
df_dpe.count()

+-------------+------------------+----------------------+--------------------------+--------------------+-----------+-----------------------+----------------------------+-----------------------+------------------------------+-------------+------------------+-------------+---------------------------+-------------------------------+--------------------+--------------------+------------------+----------------------+----------------------+--------------------------+--------------------------+--------------------------+-----------------------+------------------+-------------------------------+----------------+----------------+--------------------+--------------------+----------------+-------------+----------------+--------------------+-----------------+---------+-----------------+---------------------------------+---------------------------------+------------------+--------------------+-----------------------------+---------------+-----------------------------+--------------------+---------

338418

In [7]:
# Show how many null for each col
df_dpe.select([count(when(col(c).isNull(), c)).alias(c) for c in df_dpe.columns]).show()


+-----+------------------+----------------------+--------------------------+----------+-----------+-----------------------+----------------------------+-----------------------+------------------------------+-------------+------------------+-------------+---------------------------+-------------------------------+--------------------+--------------------+------------------+----------------------+----------------------+--------------------------+--------------------------+--------------------------+-----------------------+------------------+-------------------------------+---------------+----------------+-------------+------------------+----------------+-------------+----------------+-------------+-----------------+---------+--------------+---------------------------------+---------------------------------+------------------+--------------------+-----------------------------+---------------+-----------------------------+----------------+--------------------+---------------+--------------

In [12]:
row_count = df_dpe.count()

# Calculate the percentage of missing values for each column
null_percentage = df_dpe.select([(count(when(col(c).isNull(), c)) / row_count * 100).alias(c) for c in df_dpe.columns])
null_percentage.show()

+-----+------------------+----------------------+--------------------------+------------------+------------------+-----------------------+----------------------------+-----------------------+------------------------------+------------------+------------------+------------------+---------------------------+-------------------------------+--------------------+--------------------+------------------+----------------------+----------------------+--------------------------+--------------------------+--------------------------+-----------------------+------------------+-------------------------------+------------------+------------------+------------------+------------------+----------------+-----------------+-----------------+------------------+-----------------+-----------------+-----------------+---------------------------------+---------------------------------+------------------+--------------------+-----------------------------+-----------------+-----------------------------+----------

In [14]:
row_count = df_dpe.count()

# Calculate the percentage of missing values for each column
null_percentage = df_dpe.select([(count(when(col(c).isNull(), c)) / row_count * 100).alias(c) for c in df_dpe.columns])

# Convert to Pandas DataFrame for easier manipulation
null_percentage_pd = null_percentage.toPandas()

# Transpose and sort by percentage of missing values
null_percentage_pd = null_percentage_pd.transpose().reset_index()
null_percentage_pd.columns = ['Column', 'MissingPercentage']
null_percentage_pd = null_percentage_pd.sort_values(by='MissingPercentage', ascending=False)

# Print the list ordered by most missing lines
null_percentage_pd

,Column,MissingPercentage
140,Facteur_couverture_solaire_installation_chauff...,99.966905
152,Type_énergie_générateur_n°2_installation_n°2,99.641863
153,Usage_générateur_n°2_installation_n°2,99.641863
150,Conso_chauffage_dépensier_générateur_n°2_insta...,99.641568
151,Description_générateur_chauffage_n°2_installat...,99.641568
...,...,...
2,Date_établissement_DPE,0.683474
28,Adresse_brute,0.683179
12,Type_bâtiment,0.679042
1,Date_réception_DPE,0.659835


In [19]:

# Filter columns with less than 50% missing values
columns_less_than_50_missing = null_percentage_pd[null_percentage_pd['MissingPercentage'] < 50]['Column'].tolist()

# Print the list of columns
print(len(columns_less_than_50_missing))
columns_less_than_50_missing

110


['Surface_habitable_immeuble',
 'Conso_chauffage_é_finale_énergie_n°2',
 'Coût_chauffage_énergie_n°2',
 'Coût_total_5_usages_énergie_n°2',
 'Conso_5_usages_é_finale_énergie_n°2',
 'Type_énergie_n°2',
 'Type_ventilation',
 'Surface_ventilée',
 'Ventilation_postérieure_2012_(0/1)',
 'Appartement_non_visité_(0/1)',
 'Type_installation_ECS_(général)',
 'Nombre_niveau_immeuble',
 "Complément_d'adresse_logement",
 'Type_installation_chauffage',
 'Présence_production_PV_(0/1)',
 'N°_voie_(BAN)',
 'Nombre_appartement',
 'Année_construction',
 'Description_générateur_chauffage_n°1_installation_n°1',
 'Qualité_isolation_plancher_bas',
 'Description_installation_chauffage_n°1',
 'N°_région_(BAN)',
 'N°_département_(BAN)',
 'Besoin_refroidissement_dépensier',
 'Conso_chauffage_dépensier_générateur_n°1_installation_n°1',
 'Conso_chauffage_générateur_n°1_installation_n°1',
 'Production_électricité_PV_(kWhep/an)',
 'Type_émetteur_installation_chauffage_n°1',
 'Surface_chauffée_installation_chauffage_